In [1]:
import pandas as pd
import numpy as np
import math
import base64
import matplotlib.pyplot as plt
import seaborn as sns
from sodapy import Socrata

In [2]:
client = Socrata(
        domain = "data.cdc.gov",
        app_token = "bsYMLkKAQhIIVd7wzBzp5BiCR",
        timeout=10
)
results = client.get_all("unsk-b7fc")

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)
df.sample(10)

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_unk_manuf,dist_per_100k,distributed_per_100k_12plus,...,administered_dose1_recip_5pluspop_pct,series_complete_5plus,series_complete_5pluspop_pct,administered_5plus,admin_per_100k_5plus,distributed_per_100k_5plus,series_complete_moderna_5plus,series_complete_pfizer_5plus,series_complete_janssen_5plus,series_complete_unk_manuf_5plus
12250,2021-07-02T00:00:00.000,26,DC,1062055,54400,423240,584415,0,150486,174178,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
9279,2021-08-17T00:00:00.000,33,UT,3518080,172400,1442000,1903680,0,109736,135715,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
13976,2021-06-05T00:00:00.000,22,MH,51300,10800,40500,0,0,87823,102839,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
7221,2021-09-17T00:00:00.000,37,SC,6652745,315800,2868180,3468765,0,129212,150515,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
16892,2021-04-21T00:00:00.000,16,IH2,1666225,73000,896100,697125,0,79999,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
20698,2021-02-22T00:00:00.000,8,AR,691525,0,0,0,0,22915,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
6541,2021-09-28T00:00:00.000,39,MP,86200,2700,20200,63300,0,151542,177454,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4011,2021-11-06T00:00:00.000,44,VI,117520,1800,31840,83280,600,110565,131417,...,56.5,49474,49.7,110683,111101,117964,13109,34875,1486,4
7839,2021-09-08T00:00:00.000,36,IH2,2008065,74700,988560,944805,0,96411,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
11201,2021-07-18T00:00:00.000,29,VA,10566455,545400,4132400,5888655,0,123794,144608,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN


In [3]:
df.columns
df.dtypes

date                               object
mmwr_week                          object
location                           object
distributed                        object
distributed_janssen                object
                                    ...  
distributed_per_100k_5plus         object
series_complete_moderna_5plus      object
series_complete_pfizer_5plus       object
series_complete_janssen_5plus      object
series_complete_unk_manuf_5plus    object
Length: 80, dtype: object

In [4]:
# removing all distribution cols
# removing "Administered", total for 1st dose and complete series
# removing admin unknown manufacturer, not within project scope
# removing all 5plus cols, very recently approved and sporadic across states
# remove additional doses for 50plus, age group doesn't show up elsewhere in dataset

# df = df.drop(columns= ["MMWR_week","Distributed", "Distributed_Janssen", "Distributed_Moderna", 
#                        "Distributed_Pfizer", "Distributed_Unk_Manuf", "Dist_Per_100K","Distributed_Per_100k_12Plus",
#                        "Distributed_Per_100k_18Plus", "Distributed_Per_100k_65Plus", "Distributed_Per_100k_5Plus",
#                        "Administered", "Administered_Unk_Manuf", "Series_Complete_Unk_Manuf",
#                        "Series_Complete_Unk_Manuf_12Plus", "Series_Complete_Unk_Manuf_18Plus", "Series_Complete_Unk_Manuf_65Plus",
#                        "Additional_Doses_Unk_Manuf", "Administered_Dose1_Recip_5Plus", "Administered_Dose1_Recip_5PlusPop_Pct",
#                        "Series_Complete_5Plus", "Series_Complete_5PlusPop_Pct", "Administered_5Plus",
#                        "Admin_Per_100k_5Plus", "Series_Complete_Moderna_5Plus", "Series_Complete_Pfizer_5Plus",
#                        "Series_Complete_Janssen_5Plus", "Series_Complete_Unk_Manuf_5Plus","Additional_Doses_50Plus",
#                        "Additional_Doses_50Plus_Vax_Pct"], axis=1)
# !!!!! NEED TO SCALE TOTALED COLS FOR REMOVAL OF 5PLUS, 50PLUS AND UNK MANU !!!!!!
df = df.drop(columns= ["MMWR_week","Distributed", "Distributed_Janssen", "Distributed_Moderna", 
                       "Distributed_Pfizer", "Distributed_Unk_Manuf", "Dist_Per_100K","Distributed_Per_100k_12Plus",
                       "Distributed_Per_100k_18Plus", "Distributed_Per_100k_65Plus", "Distributed_Per_100k_5Plus",
                       "Administered","Administered_Dose1_Recip_5Plus",
                       "Administered_Dose1_Recip_5PlusPop_Pct", "Series_Complete_5Plus", "Series_Complete_5PlusPop_Pct", 
                       "Administered_5Plus", "Admin_Per_100k_5Plus", "Series_Complete_Moderna_5Plus",
                       "Series_Complete_Pfizer_5Plus", "Series_Complete_Janssen_5Plus", "Series_Complete_Unk_Manuf_5Plus",
                       "Additional_Doses_50Plus", "Additional_Doses_50Plus_Vax_Pct"], axis=1)
df.head(10)

KeyError: "['MMWR_week' 'Distributed' 'Distributed_Janssen' 'Distributed_Moderna'\n 'Distributed_Pfizer' 'Distributed_Unk_Manuf' 'Dist_Per_100K'\n 'Distributed_Per_100k_12Plus' 'Distributed_Per_100k_18Plus'\n 'Distributed_Per_100k_65Plus' 'Distributed_Per_100k_5Plus' 'Administered'\n 'Administered_Dose1_Recip_5Plus' 'Administered_Dose1_Recip_5PlusPop_Pct'\n 'Series_Complete_5Plus' 'Series_Complete_5PlusPop_Pct'\n 'Administered_5Plus' 'Admin_Per_100k_5Plus'\n 'Series_Complete_Moderna_5Plus' 'Series_Complete_Pfizer_5Plus'\n 'Series_Complete_Janssen_5Plus' 'Series_Complete_Unk_Manuf_5Plus'\n 'Additional_Doses_50Plus' 'Additional_Doses_50Plus_Vax_Pct'] not found in axis"

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
# abbreviating column names for readability 

rename_cols = {"administered_12plus":"admin_12plus", 
               "administered_18plus":"admin_18plus",
               "administered_65plus":"admin_65plus",
               "administered_janssen":"admin_janssen",
               "administered_moderna":"admin_moderna",
               "administered_pfizer":"admin_pfizer",
               "administered_unk_manuf":"admin_unk",
               "recip_administered":"recip_admin",
               "administered_dose1_recip":"admin_dose1",
               "administered_dose1_pop_pct":"admin_dose1_pop_pct",
               "administered_dose1_recip_12plus":"admin_dose1_12plus",
               "administered_dose1_recip_12pluspop_pct":"admin_dose1_12plus_pop_pct",
               "administered_dose1_recip_18plus":"admin_dose1_18plus",
               "administered_dose1_recip_18pluspop_pct":"admin_dose1_18plus_pop_pct",
               "administered_dose1_recip_65plus":"admin_dose1_65plus",
               "administered_dose1_recip_65pluspop_pct":"admin_dose1_65plus_pop_pct",
               "series_complete_yes":"series_complete",
               "series_complete_unk_manuf":"series_complete_unk",
               "series_complete_unk_manuf_12plus":"series_complete_unk_12plus",
               "series_complete_unk_manuf_18plus":"series_complete_unk_18plus",
               "series_complete_unk_manuf_65plus":"series_complete_unk_65plus",
               "series_complete_12pluspop_pct":"series_complete_12plus_pop_pct",
               "series_complete_18pluspop_pct":"series_complete_18plus_pop_pct",
               "series_complete_65pluspop_pct":"series_complete_65plus_pop_pct"
               }
df = df.rename(columns=rename_cols)

In [ ]:
# removing locations outside of the contential united states as well as united states total

drop_location = ["AS", "BP2", "DD2", "FM", "GU",
                 "IH2", "MH", "MP", "PR", "RP",
                 "US", "VA2", "VI", "LTC"]
df = df[~df["location"].isin(drop_location)]

# checking that location vals = 51, including DC

df.location.nunique()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# converting "date" column to datetime
df["date"] = pd.to_datetime(df["date"])
# df.dtypes
# df

In [ ]:
# filter location to New York
df_ny = df.loc[df["location"] == "NY"]

In [ ]:
# set index to date to extract rows for the first of each month, aligns with how other datasets are reported
df_ny_m = df_ny.set_index("date")
df_ny_m = df_ny_m.groupby(df_ny_m.index.strftime("%Y-%m")).tail(1)
df_ny_m.head(14)

In [ ]:
# first and most recent dates for vax reported data, updated daily
print(df_ny["date"].min())
print(df_ny["date"].max())

In [ ]:
# trend in administration of each vax manufacturer 
ny_m_plt = df_ny_m.reset_index()
ny_m_plt.plot(x="date", y=["admin_janssen", "admin_moderna", "admin_pfizer", "admin_unk"], kind="line")